In [7]:
# Importing libraries
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from bs4 import BeautifulSoup
from requests import get

# Creating variables for HTML tags and classes
tags = ['span','span','span','a','li']
classes = ['price-tag-fraction',
           'ui-search-item__group__element ui-search-item__location',
           'ui-search-item__group__element ui-search-item__subtitle',
           'ui-search-result__content ui-search-link',
           'ui-search-card-attributes__attribute']
                   
# Empty lists for storing the scrapped data
a_size = len(classes)
lists = [[] for i in range(a_size)]
link = []
 
# For loop to go through multiple pages and retrieve the data  
pages = np.arange(1,50,48) # The second element of this array determines "how far" our for loop will go through the pages. 
                           # In this case I set it to a pretty low number (50) only for demonstration purposes.
for page in pages:
    page = requests.get("https://imoveis.mercadolivre.com.br/aluguel/_Desde_"+str(page)+"_NoIndex_True")
    soup = BeautifulSoup(page.text, 'html.parser')
    sleep(randint(2,8))
    house_data = soup.find_all('div',class_="ui-search-item__group ui-search-item__group--price")
    for i in range(a_size):
        for data in soup.find_all(tags[i], class_=classes[i]):
            if tags[i] == 'a':
                lists[i].append(data['href'])
            else:
                lists[i].append(data.get_text())
                                            
## Creating Data Frames to treat the data ##      
df = pd.DataFrame({'preços':lists[0],
                   'localização':lists[1],
                   'tipo':lists[2],
                   'link':lists[3]})
df_atc = pd.DataFrame({'atc':lists[4]})

# House type
df[['tipo_imovel','tipo_compra']] = df['tipo'].str.split('para',expand=True)
df = df.drop(['tipo'], axis=1)  

# Location
df['uf'] = df['localização'].str.split(',').str[-1]
df['cidade'] = df['localização'].str.split(',').str[-2]

# Total area
df_atc = df_atc[df_atc['atc'].str.contains('quarto')==False]  
words_to_replace = {'m² construídos':'',
                    'm² totais':''}
for i, j in words_to_replace.items():
    df_atc['atc'] = df_atc['atc'].str.replace(i, j)
df_atc.reset_index(drop=True, inplace=True)

# Exporting to a .xlsx file
df = df.join(df_atc)
df.to_excel('webscrapping_mercadolivre.xlsx')